## Get 2010-2018 Max/Min Temps

In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from IPython.display import display 
from IPython.display import display_html
import datetime
import time

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
low_url = 'https://www.currentresults.com/Yearly-Weather/USA/CA/Los-Angeles/extreme-annual-los-angeles-low-temperature.php'
browser.visit(low_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [4]:
# Function to return DataFrame with extreme values 
def tempDataFrame(df):
    df.drop(columns = [df.columns[2]])
    df['Date'] = pd.to_datetime(df['Date'])
    years = [t.year for t in df['Date']]
    df['Year'] = years
    columns_titles = ["Year","Date", df.columns[0]]
    df = df.reindex(columns=columns_titles)
    return df

In [5]:
lowest_temps = pd.read_html(html)[2]
# #Get rid of the plus sign
lowest_temps['Date'][0] = lowest_temps['Date'][0][:-2]
lowest_temps['Date'][6] = lowest_temps['Date'][6][:-2]
low_temps = tempDataFrame(lowest_temps)
low_temps

/Users/morganivey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/morganivey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Year,Date,Min °F
0,2018,2018-02-24,40
1,2017,2017-12-22,40
2,2016,2016-12-19,40
3,2015,2015-12-27,36
4,2014,2014-12-27,39
5,2013,2013-01-14,34
6,2012,2012-12-31,39
7,2011,2011-12-06,37
8,2010,2010-12-31,37


In [6]:
high_url = 'https://www.currentresults.com/Yearly-Weather/USA/CA/Los-Angeles/extreme-annual-los-angeles-high-temperature.php'
browser.visit(high_url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [25]:
highest_temps = pd.read_html(html)[2]
high_temps = tempDataFrame(highest_temps)
high_temps

,Year,Date,Max °F
0,2018,2018-07-06,108
1,2017,2017-10-24,104
2,2016,2016-09-26,104
3,2015,2015-09-09,101
4,2014,2014-09-16,103
5,2013,2013-08-29,96
6,2012,2012-09-15,103
7,2011,2011-10-12,99
8,2010,2010-09-27,113


## Get 2019 Max/Min Temps

In [8]:
import pandas as pd
import numpy as np
import requests
import datetime as dt
from pprint import pprint
# Import API key
from api_keys import api_key
from api_keys import wwo_api_key
from datetime import timedelta, date
import csv
import time
import matplotlib.pyplot as plt

In [9]:
def daterange(start_date, end_date):
    for n in range(0,int((end_date - start_date).days)+1,34):
        yield start_date + timedelta(n)

# Get range of dates for 2018, in chunks of 34 days 
#*** Issues with API when set date range to more than 34 days
start_date = date(2019,1,1)
end_date = date(2019,12,31)
date_ranges = []
for single_date in daterange(start_date, end_date):
    date_ranges.append(single_date.strftime("%Y-%m-%d"))

#Append last date chunk:
date_ranges.append('2019-12-31')

In [10]:
# Create CSV file to hold the weather data (create empty CSV with headers only)
with open('OutputData/tempExtremes_2019.csv','w') as writeFile:
    writer = csv.DictWriter(writeFile, fieldnames = ['Date','Min Temp (F)','Max Temp (F)'])
    writer.writeheader()

# Call the weather API to get weather data for each period of 34 days
for i in range(1,len(date_ranges)):
    
    # Set start and end dates
    start_d = date_ranges[i-1]
    end_d = date_ranges[i]
    
    # Ensure it is working
    print(f'Getting {start_d} to {end_d}')
    
    # Get average daily weather
    frequency = 24

    # api_key = wwo_api_key
    location = 'Los Angeles'

    # Get weather response
    url_page = 'http://api.worldweatheronline.com/premium/v1/past-weather.ashx?key='+wwo_api_key+'&q='+location+'&format=json&date='+start_d+'&enddate='+end_d+'&tp='+str(frequency)
    response = requests.get(url_page)
    weather_response = response.json()
    
    #*** Store max and min temps of that day
    # Store dates, average temperatures, number of hours of sun, and description of weather into lists
    dates = [response['date'] for response in weather_response['data']['weather']]
    min_temps = [response['mintempF'] for response in weather_response['data']['weather']]
    max_temps = [response['maxtempF'] for response in weather_response['data']['weather']]
#     avg_temps = [response['avgtempF'] for response in weather_response['data']['weather']]
#     sun_hours = [response['sunHour'] for response in weather_response['data']['weather']]
#     weather_desc = [response['hourly'][0]['weatherDesc'][0]['value'] for response in weather_response['data']['weather']]
    
    # Create a list of tuples of above lists to write to CSV rows
    weather_zip = list(zip(dates,min_temps,max_temps))
    
    # Append each tuple to row in CSV 
    for z in weather_zip:
        with open('OutputData/tempExtremes_2019.csv','a') as writeFile:
            writer = csv.writer(writeFile)
            writer.writerow(z)

Getting 2019-01-01 to 2019-02-04
Getting 2019-02-04 to 2019-03-10
Getting 2019-03-10 to 2019-04-13
Getting 2019-04-13 to 2019-05-17
Getting 2019-05-17 to 2019-06-20
Getting 2019-06-20 to 2019-07-24
Getting 2019-07-24 to 2019-08-27
Getting 2019-08-27 to 2019-09-30
Getting 2019-09-30 to 2019-11-03
Getting 2019-11-03 to 2019-12-07
Getting 2019-12-07 to 2019-12-31


In [11]:
weather2019 = pd.read_csv('OutputData/tempExtremes_2019.csv')
# weather2019

min_temp_2019 = weather2019.loc[weather2019['Min Temp (F)'] == weather2019['Min Temp (F)'].min()]
max_temp_2019 = weather2019.loc[weather2019['Max Temp (F)'] == weather2019['Max Temp (F)'].max()]
max_temp_2019['Date'] = pd.to_datetime(max_temp_2019['Date'])
min_temp_2019['Date'] = pd.to_datetime(min_temp_2019['Date'])

# Two min temps for 2019: choose one:
min_temp_2019 = min_temp_2019.head(1)

/Users/morganivey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/morganivey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [26]:
temp2019 = max_temp_2019['Max Temp (F)'].values[0]
year2019 = max_temp_2019['Date'].iloc[0].year
date2019 = max_temp_2019['Date'].iloc[0]
date2019

high_temps = high_temps.append({'Year': year2019, 'Date': date2019,high_temps.columns[2]: temp2019}, ignore_index=True)
high_temps
high_temps = high_temps.sort_values(by="Date")
high_temps


,Year,Date,Max °F
8,2010,2010-09-27,113
7,2011,2011-10-12,99
6,2012,2012-09-15,103
5,2013,2013-08-29,96
4,2014,2014-09-16,103
3,2015,2015-09-09,101
2,2016,2016-09-26,104
1,2017,2017-10-24,104
0,2018,2018-07-06,108
9,2019,2019-09-14,98


In [98]:
temp2019 = min_temp_2019['Min Temp (F)'].values[0]
year2019 = min_temp_2019['Date'].iloc[0].year
date2019 = min_temp_2019['Date'].iloc[0]

low_temps = low_temps.append({'Year': year2019, 'Date': date2019,low_temps.columns[2]: temp2019}, ignore_index=True)
low_temps = low_temps.sort_values(by="Date")
low_temps.to_csv('OutputData/Min_Temps_2010_2019.csv')